In [16]:
from geopy.geocoders import Nominatim
import pandas as pd

In [51]:
geolocator = Nominatim()

In [21]:
data = pd.read_excel("full_data.xlsx")

In [22]:
data.head()

,Code Insee,NListe,Nuance Liste,Voix,% Voix/Ins,% Voix/Exp,REG,DEP,Libelle de la commune,P12_POP,...,SNHMHO12,SNHM1812,SNHM2612,SNHM5012,SNHMF1812,SNHMF2612,SNHMF5012,SNHMH1812,SNHMH2612,SNHMH5012
0,1001,2,LDVD,0,0.00,0.00,82,01,L'Abergement-Clemenciat,777,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,9,LDLF,7,1.20,2.19,82,01,L'Abergement-Clemenciat,777,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1001,6,LUD,100,17.12,31.25,82,01,L'Abergement-Clemenciat,777,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1002,6,LUD,27,12.05,21.26,82,01,L'Abergement-de-Varey,235,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1002,2,LDVD,0,0.00,0.00,82,01,L'Abergement-de-Varey,235,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
import unicodedata

def unicode_to_string(word):
    if pd.isnull(word):
        return 
    else:
        return unicodedata.normalize('NFKD', word).encode('ascii','ignore')
    
def data_to_string(data):
    new_data = data.copy()
    
    col_title_unicode = new_data.columns.values.tolist()
    col_title = map(unicode_to_string,col_title_unicode)
    new_data.columns = col_title
    
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) == unicode: #check the first not-NaN value
            new_data[col] = map(unicode_to_string,new_data[col])
            
    return new_data

In [46]:
data = data_to_string(data)

In [48]:
town_list = list(set(data['Libelle de la commune'].tolist()))

In [53]:
town_list[1], town_list[100], town_list[1000]

('Berthenonville', 'Ceaux', 'Flety')

In [93]:
len(town_list)

34063

In [104]:
l_lat = [] 
l_lon = [] 
for x in xrange(50):
    loc = geolocator.geocode(town_list[x])
    l_lat.append(loc.latitude)
    l_lon.append(loc.longitude)

GeocoderServiceError: HTTP Error 429: Too Many Requests

In [90]:
geolocator.geocode('Sains-en-Gohelle')

Location((50.4503, 2.68356, 0.0))

In [81]:
from bokeh.io import output_file, show
from bokeh.models import GeoJSONDataSource
from bokeh.plotting import figure
from bokeh.sampledata.sample_geojson import geojson

In [82]:
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
)

In [83]:
map_options = GMapOptions(lat=loc1.latitude, lng=loc1.longitude, map_type="hybrid", zoom=6)

plot = GMapPlot(
    x_range=DataRange1d(), y_range=DataRange1d(), map_options=map_options, title="Austin"
)

source = ColumnDataSource(
    data=dict(
        #lat=[30.29, 30.20, 30.29],
        #lon=[-97.70, -97.74, -97.78],
        #lat = [loc1.latitude, loc2.latitude, loc3.latitude], 
        lat = l_lat ,
        lon = l_lon
        #lon = [loc1.longitude, loc2.longitude, loc3.longitude]
        #lat,
        #lon
    )
)

circle = Circle(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, line_color=None)
plot.add_glyph(source, circle)

plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
output_file("gmap_plot.html")
show(plot)